In [20]:
from scipy import signal
import numpy as np
from scipy.interpolate import interp1d
import pandas as pd
import cv2
import pickle

In [21]:
# from saved data after running program
ball_x = np.load("ball_x.npy", allow_pickle = True)
ball_y = np.load("ball_y.npy", allow_pickle = True)
ball_xy = np.vstack((ball_x, ball_y)).T
ball_xy = pd.DataFrame(ball_xy)
ball_xy.to_csv("ball_xy.csv")

In [22]:
ball_pos = pd.read_csv("ball_xy.csv")
ball_pos.drop("Unnamed: 0", axis = 'columns', inplace = True)

In [23]:
ball_pos

,0,1
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,942.0,417.0
...,...,...
296,879.0,348.0
297,879.0,357.0
298,882.0,357.0
299,891.0,357.0


In [24]:
# using saved ball_positions --> would come from ball_detector.xy_coordinates in code
def calc_ball_positions(ball_positions, court_detector):
    inv_mats = court_detector.game_warp_matrix
    ball_pos = []

    ## from find_stroke_indices function
    # lots of NaNs that we want to fill in
    ball_x = ball_positions.loc[:, "0"]
    ball_y = ball_positions.loc[:, "1"]
    smooth_x = signal.savgol_filter(ball_x, 3, 2)
    smooth_y = signal.savgol_filter(ball_y, 3, 2)

    # Ball position interpolation, since lots of NaNs
    x = np.arange(0, len(smooth_y))
    indices = [i for i, val in enumerate(smooth_y) if np.isnan(val)]
    x = np.delete(x, indices)
    y1 = np.delete(smooth_y, indices)
    y2 = np.delete(smooth_x, indices)
    ball_f2_y = interp1d(x, y1, kind='cubic', fill_value="extrapolate")
    ball_f2_x = interp1d(x, y2, kind='cubic', fill_value="extrapolate")

    # following code from github, but may want to change this
    xnew = np.linspace(0, len(ball_y), num=len(ball_y), endpoint=True)
    coordinates = np.column_stack((ball_f2_x(xnew),ball_f2_y(xnew)))


    positions = []
    for i, xy_coord in enumerate(coordinates):
        xy_coord = xy_coord.reshape((1,1,2))
        # transforms data into 2D court
        ball_court_pos = cv2.perspectiveTransform(xy_coord, inv_mats[i]).reshape(-1)
        positions.append(ball_court_pos) 

    positions = np.array(positions)
    smoothed = np.zeros_like(positions)

    # puts a final interpolation filter on for smoother data in the video
    smoothed[:, 0] = signal.savgol_filter(positions[:, 0], 7, 2)
    smoothed[:, 1] = signal.savgol_filter(positions[:, 1], 7, 2)

    return smoothed



In [25]:
# saved from code
with open('court_detect_saved.pkl', 'rb') as file:
    court_detect_saved = pickle.load(file)

with open('top_view_court_ref.pkl', 'rb') as file:
    top_view_court_ref = pickle.load(file)

In [26]:
calc_ball_positions(ball_pos, court_detect_saved)

array([[ 6.39086726e+02,  3.64276771e+03],
       [ 6.37277106e+02,  3.64511130e+03],
       [ 6.35910632e+02,  3.64698006e+03],
       [ 6.34987301e+02,  3.64837397e+03],
       [ 6.34901284e+02,  3.64868118e+03],
       [ 6.34721573e+02,  3.64914587e+03],
       [ 6.34472564e+02,  3.65030562e+03],
       [ 6.34134708e+02,  3.65224854e+03],
       [ 6.33720284e+02,  3.65403826e+03],
       [ 6.33315288e+02,  3.65607226e+03],
       [ 6.32877400e+02,  3.65843921e+03],
       [ 6.32407501e+02,  3.66118017e+03],
       [ 6.31902077e+02,  3.66437217e+03],
       [ 6.31357100e+02,  3.66811213e+03],
       [ 6.30767945e+02,  3.67252312e+03],
       [ 6.30129277e+02,  3.67776301e+03],
       [ 6.29434923e+02,  3.68403657e+03],
       [ 6.28671821e+02,  3.69169202e+03],
       [ 6.27838354e+02,  3.70066462e+03],
       [ 6.26996441e+02,  3.71192417e+03],
       [ 6.26007057e+02,  3.72648716e+03],
       [ 6.24922793e+02,  3.74466575e+03],
       [ 6.23632403e+02,  3.76827196e+03],
       [ 6.

In [31]:
def create_top_view(court_detector, ball_pos):
    # taken from process.py function
    """
    Creates top view video of the gameplay
    """
    court = court_detector.court_reference.court.copy()
    court = cv2.line(court, *court_detector.court_reference.net, 255, 5)
    v_width, v_height = court.shape[::-1]
    court = cv2.cvtColor(court, cv2.COLOR_GRAY2BGR)


    out = cv2.VideoWriter('output/top_view_ball.avi',
                          cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (v_width, v_height))
    
    smoothed = calc_ball_positions(ball_pos, court_detect_saved)

    for ball_pos_elem in smoothed:
        frame = court.copy()
        frame = cv2.circle(frame, (int(ball_pos_elem[0]), int(ball_pos_elem[1])), 10, (52, 225, 235), 15)

        out.write(frame)
        
    out.release()
    cv2.destroyAllWindows()

In [32]:
create_top_view(court_detect_saved, ball_pos)